In [1]:
# TO DO
# Оформить describe
# В pair plot разбивка по факультетам

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('datasets/dataset_train.csv', index_col = 0)
df = df.fillna(df.mean())

df['Gryffindor'] = np.where(df['Hogwarts House'] ==  'Gryffindor', 1, 0)
df['Slytherin'] = np.where(df['Hogwarts House'] ==  'Slytherin', 1, 0)
df['Ravenclaw'] = np.where(df['Hogwarts House'] ==  'Ravenclaw', 1, 0)
df['Hufflepuff'] = np.where(df['Hogwarts House'] ==  'Hufflepuff', 1, 0)

X = df[['Astronomy', 'Herbology', 'Divination', 'Ancient Runes', 'Charms', 'Flying']]
X = preprocessing.normalize(X) #X = (X - X.min()) / (X.max() - X.min())

y = df[['Gryffindor', 'Slytherin', 'Ravenclaw', 'Hufflepuff']]

X, X_test, y, y_test = train_test_split(X, y)

X = np.array(X)
#y = np.array(y)
X_test = np.array(X_test)
#y_test = np.array(y_test)

X = np.concatenate((X, np.ones([1200, 1])), axis = 1) 
X_test = np.concatenate((X_test, np.ones([400, 1])), axis = 1) 
X

array([[-7.85243335e-01, -4.20385742e-03, -3.50014660e-03, ...,
        -3.19652566e-01, -7.09559746e-02,  1.00000000e+00],
       [-8.03589124e-01, -7.82059337e-03, -7.65421099e-03, ...,
        -3.18856625e-01, -9.21532626e-02,  1.00000000e+00],
       [ 5.60987586e-01, -8.23206014e-03,  5.07144063e-03, ...,
        -3.56876562e-01,  2.74255774e-01,  1.00000000e+00],
       ...,
       [ 6.13161348e-01, -3.36867045e-03,  8.90048178e-03, ...,
        -2.83602670e-01,  2.25134538e-01,  1.00000000e+00],
       [-6.90877438e-01,  8.13127827e-03,  4.33455356e-03, ...,
        -2.66143500e-01, -6.13926614e-04,  1.00000000e+00],
       [-5.88077089e-01,  3.44006950e-03,  6.93984872e-03, ...,
        -2.66119114e-01, -2.88824743e-03,  1.00000000e+00]])

In [4]:
# TRAIN

lr = 0.5

W_matrix = []
houses = ['Gryffindor', 'Slytherin', 'Ravenclaw', 'Hufflepuff']

for house in houses:
    y_train = y[house]
    y_train = np.array(y_train)
    
    theta = np.zeros([1, 7]).ravel()
    while True:
        grad = np.zeros(theta.shape) #(7,)
        predict = 1 / (1 + np.exp(-np.dot(X, theta.T)))
        error = (predict - y_train).ravel()

        for j in range(len(theta)): # for each parmeter
            term = np.multiply(error, X[:,j]) # поэлементное умножение
            grad[j] = np.sum(term) / len(X)
        new_theta = theta - lr * grad

        if (np.sum(theta - new_theta) ** 2) ** 0.5 < 0.0001:
            break
        theta = new_theta
    W_matrix.append(new_theta)
    
print(W_matrix)
    

[array([ 2.44017567, -4.69547616,  0.08781753, 12.78574531, 15.5941881 ,
       34.52496902, -9.62545815]), array([ -6.88039284, -13.23524135, -12.96939313, -17.25026475,
       -41.19647366,  -9.7751314 ,  -7.18118923]), array([-2.10541807,  0.04326435,  0.03129442,  0.02993638,  0.62143272,
        0.07529138, -0.96053326]), array([ 0.28748195,  0.0023754 ,  0.00070613, -0.1601746 ,  0.04849115,
       -0.02234336, -0.19650785])]


In [5]:
# TEST
# houses = {'Gryffindor': 0, 'Slytherin': 1, 'Ravenclaw': 2, 'Hufflepuff': 3}

df_result = pd.DataFrame(y_test)
for house in houses:
    if house == 'Gryffindor':
        index = 0
    elif house == 'Slytherin':
        index = 1
    elif house == 'Ravenclaw':
        index = 2
    else:
        index = 3
    theta = W_matrix[index]
    
    predict = (1 / (1 + np.exp(-np.dot(X_test, theta.T)))).ravel()
    df_result[index] = predict
    
df_result.to_excel('./RESULT.xlsx')


In [6]:
df_result['max'] = df_result[[0, 1, 2, 3]].max(axis=1)

#df_result.where(df_result[0] == df_result['max'])
df_result.to_excel('RESULT.xlsx')